In [1]:
"""
Since constraints and rules are essentially the same thing, they should be unified into the same type of object. 
Things to consider in this process:

    - The Dwarf Planet constraint is similar to, but not the same as a Band Rule. A Band Rule does not require the
        objects to be at the two ends of the band; they can be anywhere within. It's not an "exact band" but a 
        maximum requirement
    - The Comet constraint is not modeled by any of the rules. A "restricted positions" type rules should be made 
        to accommodate this
    - This will allow the generate_rule/eliminate_sectors methods to take in a set of constraints and compare it to
        themselves to determine if generating a certain rule makes sense
    - It might make sense to unify SelfRule and RelationRule in the process
        - The three types of self rules are equivalent to the three types of relation rules
        - However, the comet-type rule would not be equivalent to any relation rule that I can think of
    - Constraints always had qualifiers of "none" or "every", not "at least one", which affected the fill_board 
        method since it could make fewer assumptions.
"""

'\nSince constraints and rules are essentially the same thing, they should be unified into the same type of object. \nThings to consider in this process:\n\n    - The Dwarf Planet constraint is similar to, but not the same as a Band Rule. A Band Rule does not require the\n        objects to be at the two ends of the band; they can be anywhere within. It\'s not an "exact band" but a \n        maximum requirement\n    - The Comet constraint is not modeled by any of the rules. A "restricted positions" type rules should be made \n        to accommodate this\n    - This will allow the generate_rule/eliminate_sectors methods to take in a set of constraints and compare it to\n        themselves to determine if generating a certain rule makes sense\n    - It might make sense to unify SelfRule and RelationRule in the process\n        - The three types of self rules are equivalent to the three types of relation rules\n        - However, the comet-type rule would not be equivalent to any relation

In [2]:
import itertools
import random
import json
from enum import Enum
from abc import *

In [3]:
from server.utilities import permutations_multi, fill_no_touch, fill_no_within, fill_no_self_touch

In [4]:
from server.board import *
from server.board_type import *

In [5]:
from server.game import RuleQualifier

In [6]:
class Rule(ABC):
    @abstractmethod
    def is_satisfied(self, board):
        """
        Checks whether a board meets this constraint. Returns true if constraint is met.
        """
        pass
    
    @abstractmethod
    def is_immediately_limiting(self):
        """
        Returns true if this constraint limits the positions the space object can be in
        even when no objects are on the board.
        """
        pass

    @abstractmethod
    def disallowed_sectors(self):
        """
        If the constraint is immediately limited, returns the list of sector that the 
        space object relevant to the constraint is not allowed to be in. Otherwise,
        returns [].
        """
        return []
    
    @abstractmethod
    def fill_board(self, board, num_objects):
        """
        Given a board and the number of each space object that must appear on it,
        fills the board in all ways possible given this constraint.
        
        board: A partially filled Board
        num_objects: A dictionary mapping space objects to the number of times they
            are supposed to appear in the board.
            
        Returns: a list of boards that contain the same objects as the original board,
            plus filling in objects to meet this constraint. Returns all possible
            such boards.
        """
        pass
    
    @abstractmethod
    def affects(self):
        """
        Returns a list of space objects that are affected by this rule. Note that this is
        not the same as all of the space objects named in the constraint - e.g.
            "All gas clouds are next to an empty sector" does not affect empty sectors,
            since the other empty sectors may be wherever they like
        """
        pass
    
    @abstractmethod
    def completes(self):
        """
        Returns a list of all space object that this constraint can "complete" - that is, 
        when generating all possible boards using fill_board, this constraint will add in 
        every object of that type.
        """
        pass
    
    @abstractmethod
    def adds(self):
        """
        Returns a list of all space objects that can be added during fill_board
        """
        pass
    
    @abstractmethod
    def space_objects(self):
        """
        Returns all the space objects involved in this rule
        """
        pass
    
    @classmethod
    @abstractmethod
    def generate_rule(cls, board, constraints, *space_objects):
        """
        Generates a rule of this type for a particular board and space objects to relate 
        to each other. Returns None if no such rule exists, or if such a rule would be 
        redundant with the given constraints.
        """
        pass
    
    @abstractmethod
    def code(self):
        """
        Return compressed string code that represents this rule
        """
        pass
    
    @classmethod
    @abstractmethod
    def parse(cls, s):
        """
        Parse a compressed string representation s into a rule of this type
        """
        pass
    
    @abstractmethod
    def text(self, board):
        """
        Returns a readable text version of this rule given an input Board board.
        """
        pass
    
    @abstractmethod
    def to_json(self, board):
        """
        Create a json representation of this rule, given a particular board. The json
        representation should have the following fields:
            - ruleType: type of the rule
            - If the rule is a self-rule:
                - spaceObject: the space object in the rule
            - If the rule is a relation-rule:
                - spaceObject1: the space object in the rule
                - spaceObject2: the space object which spaceObject1 is related to in the rule
            - categoryName: the name of the category the rule is in (based on types of objects)
            - text: a readable text representation of the rule
            - other rule-specific fields   
        """
        pass

In [7]:
class RelationRule(Rule):
    def space_object1(self):
        return self.space_object1
    
    def space_object2(self):
        return self.space_object2
    
    def space_objects(self):
        return [self.space_object1(), self.space_object2()]
    
    @classmethod
    @abstractmethod
    def eliminate_sectors(cls, board, data, space_object1, space_object2):
        """
        Create a rule that will eliminate possible positions of space_object1, where 
        space_object1 and data.elimination_object are ambiguous on survey/target
        
        Only the sectors in data.need_eliminated are ambiguous, and the sectors in 
        data.already_eliminated have been eliminated by other rules. To be viable,
        this rule must eliminate at least data.minimum sectors, and if possible 
        should eliminate data.goal sectors.
        """
        pass

In [8]:
class SelfRule(Rule):
    def space_object(self):
        return self.space_object
    
    def space_objects(self):
        return [self.space_object()]

In [325]:
class AdjacentSelfRule(SelfRule):
    """
    A rule stating that an object is or is not adjacent to another of the same object
    """
    def __init__(self, space_object, qualifier):
        self.space_object = space_object
        self.qualifier = qualifier
        
    def __repr__(self):
        return "<" + self.qualifier.name + " " + repr(self.space_object) + " adjacent to " \
                + repr(self.space_object) + ">"
    
    def __str__(self):
        return str(self.qualifier) + " " + self.space_object.name() + " is adjacent to another " + \
                self.space_object.name() + "."
    
    def text(self, board):
        num_object = board.num_objects()[self.space_object]
        return self.qualifier.for_object(self.space_object, num_object) + " adjacent to another " \
                + self.space_object.name() + "."
   
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.qualifier == other.qualifier and self.space_object == other.space_object
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    def is_satisfied(self, board):
        adjacent_idxs = [i for i in range(len(board)) if board[i] is self.space_object
                            and (board[i-1] is self.space_object or board[i+1] is self.space_object)]
        
        if self.qualifier is RuleQualifier.NONE:
            return len(adjacent_idxs) == 0
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            return len(adjacent_idxs) > 0
        else:
            if self.space_object in board.num_objects():
                num_obj = board.num_objects()[self.space_object]
            else:
                num_obj = 0
            return len(adjacent_idxs) == num_obj
    
    def is_immediately_limiting(self):
        return False

    def disallowed_sectors(self):
        return []
    
    def _fill_board_none(self, board, num_objects):
        if not self.is_satisfied(board):
            # There are already two adjacent in this board, cannot meet rule
            return []
        
        num_obj = num_objects[self.space_object]  
        num_obj -= sum(obj is self.space_object for obj in board)
        
        board_perms = fill_no_self_touch(self.space_object, num_obj, board)
        return [Board(board_objects) for board_objects in board_perms]
    
    def _fill_board_runs(self, board, num_obj, start_i=0):         
        # Fill in board with runs of asteroids, starting new runs only at start_i and after
        
        # If there are no asteroids left, check if board is valid
        if num_obj == 0:
            if self.is_satisfied(board):
                return [board]
            else:
                return []
        
        # If there is a lone asteroid, find it and immediately add another asteroid clockwise
        for i in range(start_i - 1, len(board)):
            obj = board[i]
            if obj is self.space_object and board[i-1] is not self.space_object \
            and board[i+1] is not self.space_object:
                # Found a lone asteroid
                new_boards = []
                
                # Only fill asteroid runs to the right without combining runs
                if board[i+1] is None and board[i+2] is not self.space_object:
                    board_copy = board.copy()
                    board_copy[i+1] = self.space_object
                    new_boards.extend(self._fill_board_runs(board_copy, num_obj - 1, start_i))
                    
                return new_boards
            
        new_boards = []
        
        for i in range(len(board)):
            obj = board[i]
            if obj is None:
                # Continue an asteroid run without combining two runs
                if board[i-1] is self.space_object and board[i+1] is not self.space_object:
                    board_copy = board.copy()
                    board_copy[i] = self.space_object
                    new_boards.extend(self._fill_board_runs(board_copy, num_obj - 1, start_i))
                # OR start a new asteroid run, if conditions allow
                elif i >= start_i and num_obj > 1 and board[i-1] is not self.space_object \
                and board[i+1] is not self.space_object:
                    board_copy = board.copy()
                    board_copy[i] = self.space_object
                    new_boards.extend(self._fill_board_runs(board_copy, num_obj - 1, i+1))
        
        return new_boards
    
    def _prepare_board(self, board, num_obj, lone, run_backwards, start_i=None):
        if start_i is None:
            start_i = len(board) - 1
        
        board_ready = len(lone) == 0 and len(run_backwards) == 0
        if board_ready:
            return [ (num_obj, board) ]

        if start_i <= 0:
            return []
        
        new_boards = []
        for i in range(start_i, -1, -1):
            obj = board[i]
            if obj is self.space_object:
                if board[i-1] is not self.space_object and board[i+1] is not self.space_object:
                    if board[i+1] is None and board[i+2] is not self.space_object:
                        board_copy = board.copy()
                        board_copy[i+1] = self.space_object
                        new_lone = lone - {i}
                        new_boards.extend(self._prepare_board(board_copy, num_obj - 1, new_lone, run_backwards, i-1))
                        
                if board[i-1] is None:
                    new_run_backwards = run_backwards - { i }
                    if board[i+1] is self.space_object:
                        new_boards.extend(self._prepare_board(board.copy(), num_obj, lone, new_run_backwards, i-1))
                        
                    num_left = num_obj
                    j = i - 1
                    board_copy = board.copy()
                    while num_left > 0:
                        if board[j] is None:
                            board_copy[j] = self.space_object
                            num_left -= 1
                            new_lone = lone - { j - 1 % len(board) }
                            new_boards.extend(self._prepare_board(board_copy, num_left, new_lone, new_run_backwards, j-1))
                            board_copy = board_copy.copy()
                            j -= 1
                        elif board[j] is self.space_object:
                            j -= 1
                        else:
                            break
                        
        return new_boards
                        
    def _fill_board_every(self, board, num_objects):
        num_left = num_objects[self.space_object] - sum(obj is self.space_object for obj in board)
        lone = set(i for i, obj in enumerate(board) if obj is self.space_object
                and board[i-1] is not self.space_object and board[i+1] is not self.space_object)
        run_backwards = set(i for i, obj in enumerate(board) if obj is self.space_object
                           and board[i-1] is None)
        boards = self._prepare_board(board, num_left, lone, run_backwards)
        new_boards = []
        for num_obj, board in boards:
            new_boards.extend(self._fill_board_runs(board, num_obj))
        return new_boards
    
    def fill_board(self, board, num_objects):
        if self.qualifier is RuleQualifier.NONE:
            return self._fill_board_none(board, num_objects)
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            # Not yet supported
            return None
        else:  
            return self._fill_board_every(board, num_objects)
            
    def affects(self):
        if self.qualifier is RuleQualifier.NONE:
            return [ self.space_object ]
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            return []
        else:
            return [ self.space_object ]
    
    def completes(self):
        if self.qualifier is RuleQualifier.NONE:
            return [ self.space_object ]
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            return []
        else:
            return [ self.space_object ]
    
    def adds(self):
        return [ self.space_object ]
    
    @classmethod
    def generate_rule(cls, board, constraints, space_object):
        # Some constraints already limit this significantly and would be redundant
        if any((isinstance(constraint, cls) and constraint == cls(space_object, constraint.qualifier))
               or (isinstance(constraint, SectorRule) and constraint.space_object == space_object) 
               for constraint in constraints):
            return None
        
        num_obj = board.num_objects()[space_object]
        
        # If there's only one object it can never be adjacent to itself
        if num_obj == 1:
            return None
        
        num_adjacent = 0
        
        # Count how many objects are adjacent
        for i, obj in enumerate(board):
            if obj is space_object:
                if board[i-1] is space_object or board[i+1] is space_object:
                    num_adjacent += 1
        
        
        # Not using every, too powerful
        if num_adjacent == 0:
            qualifier_options = [RuleQualifier.NONE]
        else:
            qualifier_options = [RuleQualifier.AT_LEAST_ONE]
        
        # At least one would mean every for these cases
        if num_obj <= 2:
            qualifier_options = [option for option in qualifier_options \
                                if option is not RuleQualifier.AT_LEAST_ONE]
            
        if len(qualifier_options) == 0:
            return None
        
        # Choose a random rule
        qualifier = random.choice(qualifier_options)
        return AdjacentSelfRule(space_object, qualifier)
    
    def code(self):
        return "C" + str(self.space_object) + self.qualifier.code()
    
    @classmethod
    def parse(cls, s):
        space_object = SpaceObject.parse(s[1])
        qualifier = RuleQualifier.parse(s[2])
        return cls(space_object, qualifier)
    
    def to_json(self, board):
        return {
            "ruleType": "ADJACENT_SELF",
            "spaceObject": self.space_object.to_json(),
            "qualifier": self.qualifier.to_json(),
            "categoryName": self.category_name(),
            "text": self.text(board)
        }

In [326]:
b = Board([SpaceObject.Asteroid, SpaceObject.Asteroid, None, SpaceObject.Asteroid, SpaceObject.Asteroid, None, None, None, None])

In [327]:
a = AdjacentSelfRule(SpaceObject.Asteroid, RuleQualifier.EVERY)

In [328]:
a.fill_board(b, {SpaceObject.Asteroid: 6})

[<Board AA-AAAA-->,
 <Board AA-AA-AA->,
 <Board AA-AAA--A>,
 <Board AA-AA--AA>,
 <Board AAAAAA--->,
 <Board AAAAA---A>]

In [117]:
b = Board([SpaceObject.Empty, SpaceObject.Comet, SpaceObject.Empty, SpaceObject.Asteroid, SpaceObject.Empty, \
           SpaceObject.Asteroid, SpaceObject.PlanetX, SpaceObject.DwarfPlanet, SpaceObject.DwarfPlanet, \
           SpaceObject.DwarfPlanet, SpaceObject.Asteroid, SpaceObject.Asteroid])

In [ ]:
from server.game import EliminationData 
data = EliminationData(SpaceObject.Empty, 1, 1, {0, 2, 4}, set())

In [ ]:
WithinRule.eliminate_sectors(SpaceObject.PlanetX, SpaceObject.Comet, data, b, [WithinRule(SpaceObject.PlanetX, SpaceObject.Comet, RuleQualifier.NONE, 2)])